In [1]:
# !pip install networkx

In [1]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

dframe = pd.read_csv('/Users/alan/Desktop/network/imdb_all_v2.csv')
gender_frame = pd.read_csv('/Users/alan/Desktop/network/gender.csv')

star_list = []
star_id_list = []
for i in range(0, len(dframe)):
    try:
        star_split = dframe['star'][i].split(',')
        id_split = dframe['star_id'][i].split(',')
        if len(star_split) == len(id_split):
            star_list += star_split
            star_id_list += id_split
    except:
        pass

In [2]:
print(len(star_list))
print(len(star_id_list))

39194
39194


In [3]:
name_id = pd.DataFrame(data = [star_list, star_id_list]).T
name_id = name_id.drop_duplicates()
name_id.columns = ['name', 'id']
name_id = name_id.set_index('id')
gender_frame = gender_frame.set_index('id')
print(len(name_id))
print(len(gender_frame))

14950
15053


In [4]:
name_id_gender = name_id.join(gender_frame, how = 'outer', on = 'id')
name_id_gender = name_id_gender.drop_duplicates('name')
name_id_gender = name_id_gender.set_index('name')
name_id_gender.head()

,id,gender
name,,
Tom Holland,nm4043618,1.0
Zendaya,nm3918035,2.0
Benedict Cumberbatch,nm1212722,1.0
Jacob Batalon,nm8188622,1.0
Timothée Chalamet,nm3154303,1.0


In [5]:
df = dframe[(2010 <= dframe['year']) & (dframe['year'] <= 2020)]
df = df.sort_values('num_votes', ascending = False).reset_index(drop = True)
df = df.iloc[0:100].reset_index(drop = True)

In [6]:
edges = []
nodes = []
for i in range(0, len(df)):
    try:
        this_star_list = df['star'][i].split(',')
        if len(this_star_list) == 4:
            nodes += this_star_list
            for m in range(len(this_star_list)):
                for n in range(m + 1, len(this_star_list)):
                    edges.append((this_star_list[m], this_star_list[n]))
    except:
        pass

G = nx.Graph(edges)
pos = nx.kamada_kawai_layout(G)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.3, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
node_adjacencies = []
node_gender = []
node_text = []

for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    try:
        node_gender.append(name_id_gender.loc[adjacencies[0]]['gender'])
            
    except:
        node_gender.append(None)
    node_text.append(str(adjacencies[0]) + ': ' + str(len(adjacencies[1])) + ' co-occurences')
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
#         colorscale='YlGnBu',
        colorscale='Bluered',
#         reversescale=True,
        color = node_gender,
        opacity = 0.9,
        size = node_adjacencies,
#         colorbar=dict(
#             thickness=15,
#             title='Star Co-occurences',
#             xanchor='left',
#             titleside='right',
#         ),
        line_width = 0))
    
# node_trace.marker.color = 
node_trace.text = node_text   

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Movie Star Co-occurences',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                plot_bgcolor='rgba(0,0,0,0)',
#                 annotations=[ dict(
#                     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
#                     showarrow=False,
#                     xref="paper", yref="paper",
#                     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()